# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2023-11-26 07:53:58.813565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 07:53:58.814011: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 07:53:58.818087: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 07:53:59.141836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 07:54:02.605321: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_final_train_old/")
ds_val = tf.data.Dataset.load("../data/ds_final_val_old/")
ds_test = tf.data.Dataset.load("../data/ds_final_test_old/")

2023-11-26 07:54:07.692657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-26 07:54:08.170171: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-26 07:54:08.170248: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-26 07:54:08.180193: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-26 07:54:08.180300: I tensorflow/compile

In [3]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3304118, 465865, 101377)

In [4]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [5]:
def transform_element(features, context, labels):
    # transform lebels from (1,) to (2,)
    labels = tf.cast(labels, tf.int32)
    labels = tf.one_hot(labels, depth=2)
    return features, labels[0]

In [6]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [7]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(y.shape)

(1024, 64, 8)
(1024, 2)


In [8]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(100, n_embed // 8)
        self.embed_dd = tf.keras.layers.Embedding(100, n_embed // 8)
        self.lstm_embed = tf.keras.layers.LSTM(n_embed // 8 * 6, return_sequences=True)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.dense2 = tf.keras.layers.Dense(2, activation='softmax')
        self.conc = tf.keras.layers.Concatenate(axis=-1)

    def call(self, x):
        cont, plate, dd = x[:,:,:-2], x[:,:,-2], x[:,:,-1] 
        x1 = self.lstm_embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x = self.conc([x1, x2, x3])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [14]:
model = MyModel(128)

In [15]:
model.build(input_shape=(None, BLOCK_SIZE, 8))
model.count_params()

322690

In [17]:
history = model.fit(ds_train, epochs=30, validation_data=ds_val)

Epoch 1/30


2023-11-26 07:54:54.973660: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 73 of 2000
2023-11-26 07:55:05.064612: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 142 of 2000
2023-11-26 07:55:25.072808: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 287 of 2000
2023-11-26 07:55:44.995282: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 425 of 2000
2023-11-26 07:55:55.766740: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 502 of 2000
2023-11-26 07:56:14.959483: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3227/3227 [==============================] - 786s 129ms/step - loss: 0.1729 - accuracy: 0.9449 - f1_score: 0.6346 - val_loss: 0.2111 - val_accuracy: 0.9424 - val_f1_score: 0.6112
Epoch 2/30


2023-11-26 08:07:55.745357: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 111 of 2000
2023-11-26 08:08:15.755644: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 280 of 2000
2023-11-26 08:08:25.769508: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 372 of 2000
2023-11-26 08:08:45.794737: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 559 of 2000
2023-11-26 08:09:05.775764: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 746 of 2000
2023-11-26 08:09:25.762672: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a 

3227/3227 [==============================] - 579s 125ms/step - loss: 0.1603 - accuracy: 0.9475 - f1_score: 0.6607 - val_loss: 0.2117 - val_accuracy: 0.9427 - val_f1_score: 0.6134
Epoch 3/30


...

3227/3227 [==============================] - 1220s 319ms/step - loss: 0.0795 - accuracy: 0.9727 - f1_score: 0.8616 - val_loss: 0.3407 - val_accuracy: 0.9288 - val_f1_score: 0.6577
Epoch 29/30


2023-11-26 16:53:36.754195: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 37 of 2000
2023-11-26 16:53:56.736121: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 161 of 2000
2023-11-26 16:54:06.753245: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 252 of 2000
2023-11-26 16:54:26.762680: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 398 of 2000
2023-11-26 16:54:36.788381: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 473 of 2000
2023-11-26 16:54:56.667107: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3227/3227 [==============================] - 765s 157ms/step - loss: 0.0765 - accuracy: 0.9736 - f1_score: 0.8665 - val_loss: 0.3564 - val_accuracy: 0.9269 - val_f1_score: 0.6550
Epoch 30/30


2023-11-26 17:06:21.926575: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 69 of 2000
2023-11-26 17:06:32.008665: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 123 of 2000
2023-11-26 17:06:51.908895: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 246 of 2000
2023-11-26 17:07:01.926922: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 316 of 2000
2023-11-26 17:07:11.938358: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 389 of 2000
2023-11-26 17:07:31.955078: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3227/3227 [==============================] - 827s 167ms/step - loss: 0.0742 - accuracy: 0.9745 - f1_score: 0.8722 - val_loss: 0.3542 - val_accuracy: 0.9304 - val_f1_score: 0.6667


In [18]:
model.evaluate(ds_test)

100/100 [==============================] - 18s 169ms/step - loss: 0.0951 - accuracy: 0.9738 - f1_score: 0.6444


[0.09505032002925873,
 0.973800778388977,
 array([0.98664975, 0.30215448], dtype=float32)]

In [19]:
model.save("../models/model_v1.keras")

In [20]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v1.csv", index=False)